In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
from functools import lru_cache
from multiprocessing import Pool

In [2]:
path_to_data = Path('data')
path_to_data

PosixPath('data')

In [3]:
%%time

path_to_pickled_data = path_to_data / 'dataframe.pickle'

if path_to_pickled_data.exists():

    df = pd.read_pickle(path_to_pickled_data)

else:
    
    class ParseError(Exception):
        pass

    @lru_cache(maxsize=2)
    def gen_columns(n):
        return ['start %', 'repeat'] + list(range(n))

    def load_file(path):

        matched = re.search('data/(.+?).csv', str(path)) # extract fraction from filename
        if not matched:
            return None


        frac = int(float(matched.group(1))*100)
        data = np.loadtxt(path, dtype=int, delimiter=',')

        # columns to join to data
        repeats = (np.arange(data.shape[0]) + 1)
        fracs = np.full(data.shape[0], frac)

        repeats.shape = fracs.shape = (data.shape[0], 1)
        df_data = np.hstack([fracs, repeats, data])

        cols = gen_columns(data.shape[1])

        df = pd.DataFrame(df_data, columns=cols)

        return df

    df = pd.DataFrame()

    pool = Pool()
    all_dfs = pool.map(load_file, path_to_data.iterdir())

    df = df.append(all_dfs)
    
    df.to_pickle(path_to_pickled_data)

df

CPU times: user 475 ms, sys: 1.13 s, total: 1.61 s
Wall time: 1.61 s


,start %,repeat,0,1,2,3,4,5,6,7,...,19991,19992,19993,19994,19995,19996,19997,19998,19999,20000
0,36,1,3661,3727,3095,3077,2903,2809,2641,2529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,2,3594,3691,3174,3088,2829,2732,2607,2503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36,3,3558,3715,3178,3049,2852,2765,2649,2494,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36,4,3651,3692,3184,3086,2923,2817,2689,2580,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36,5,3680,3681,3190,3187,2892,2798,2657,2567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6,96,592,129,87,83,81,78,77,80,...,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0
96,6,97,585,101,63,60,64,64,70,62,...,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0
97,6,98,560,124,86,85,86,84,90,88,...,52.0,52.0,52.0,52.0,52.0,52.0,52.0,52.0,52.0,52.0
98,6,99,624,154,96,92,85,82,84,90,...,124.0,124.0,124.0,124.0,124.0,124.0,124.0,124.0,124.0,124.0


In [4]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...